In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from datetime import datetime, timedelta

In [2]:
data = pd.read_csv('../Dataframe/GameWithLabels.csv')

In [3]:
def frames_to_timestamp(frame):
    seconds = (frame - 287) / 30
    a =  datetime(1, 1, 1) + pd.Timedelta(seconds=seconds)
    return a.strftime('%H:%M:%S.%f')


# Anwenden der Funktion auf den DataFrame und Hinzufügen einer neuen Spalte 'timestamp'
data['timestamp'] = data['frame'].apply(frames_to_timestamp)
data = data.set_index('timestamp')

In [4]:
data.head(5)

,frame,new_x_Player1,new_x_Player2,new_y_Player1,new_y_Player2,new_z_Player1,new_z_Player2,vx_Player1,vx_Player2,xy_Player1,...,height_Player1,height_Player2,length_Player1,length_Player2,Status,Schlag Spieler 1,Schlag Spieler 2,Satz,geschwindigkeit_p1,geschwindigkeit_p2
timestamp,,,,,,,,,,,,,,,,,,,,,
00:00:00.000000,287,0.488342,-2.556822,-1.157367,5.167203,-9.915421,11.479640,0.01540,-0.06328,-0.04420,...,1.99435,1.17692,0.64888,0.61570,Aktiv,Anschlag,NaN,1.0,0.161013,0.190461
00:00:00.033333,288,0.487970,-2.551163,-1.155410,5.141961,-9.916995,11.366557,0.02580,-0.05046,-0.03045,...,1.97376,1.17656,0.66799,0.61899,Aktiv,NaN,NaN,1.0,0.116067,0.151878
00:00:00.066666,289,0.487290,-2.544400,-1.155982,5.116309,-9.917908,11.280055,0.02339,-0.06145,-0.02760,...,1.95456,1.17893,0.68904,0.62047,Aktiv,NaN,NaN,1.0,0.105206,0.184936
00:00:00.100000,290,0.482031,-2.544177,-1.156393,5.107507,-9.918351,11.270463,0.02728,-0.04928,-0.03218,...,1.95276,1.18399,0.71109,0.62504,Aktiv,NaN,NaN,1.0,0.122710,0.148308
00:00:00.133333,291,0.477606,-2.544681,-1.157121,5.102722,-9.919782,11.268751,0.01641,-0.04386,-0.01936,...,1.94945,1.17943,0.73265,0.62681,Aktiv,NaN,NaN,1.0,0.073820,0.132001


In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

# Zeichnen der Geschwindigkeit für Spieler 1 im ersten Subplot
axs[0].plot(data.index, data['geschwindigkeit_p1'], color='blue')
axs[0].set_title('Geschwindigkeit Spieler 1')
axs[0].set_xlabel('Zeit (ms)')
axs[0].set_ylabel('Geschwindigkeit (m/s)')

# Zeichnen der Geschwindigkeit für Spieler 2 im zweiten Subplot
axs[1].plot(data.index, data['geschwindigkeit_p2'], color='green')
axs[1].set_title('Geschwindigkeit Spieler 2')
axs[1].set_xlabel('Zeit (ms)')
axs[1].set_ylabel('Geschwindigkeit (m/s)')

# Anpassung des Layouts für eine bessere Darstellung
plt.tight_layout()

# Anzeigen der Diagramme
plt.show()

In [ ]:
data["geschwindigkeit_p1_rollingMean"] = data["geschwindigkeit_p1"].rolling(30, min_periods=8).median()
data["geschwindigkeit_p2_rollingMean"] = data["geschwindigkeit_p2"].rolling(30, min_periods=8).median()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
# Zeichnen der Geschwindigkeit für Spieler 1 im ersten Subplot
axs[0,0].plot(data.index, data['geschwindigkeit_p1'], color='blue')
axs[0,0].set_title('Geschwindigkeit Spieler 1')
axs[0,0].set_xlabel('Frames (0.5 Sekunden)')
axs[0,0].set_ylabel('Geschwindigkeit (m/s)')

# Zeichnen der Geschwindigkeit für Spieler 2 im zweiten Subplot
axs[1,0].plot(data.index, data['geschwindigkeit_p2'], color='green')
axs[1,0].set_title('Geschwindigkeit Spieler 2')
axs[1,0].set_xlabel('Frames (0.5 Sekunden)')
axs[1,0].set_ylabel('Geschwindigkeit (m/s)')

# Zeichnen der Geschwindigkeit für Spieler 1 im ersten Subplot
axs[0,1].plot(data.index, data['geschwindigkeit_p1_rollingMean'], color='blue')
axs[0,1].set_title('Geschwindigkeit Spieler 1, Rolling Mean')
axs[0,1].set_xlabel('Frames (0.5 Sekunden)')
axs[0,1].set_ylabel('Geschwindigkeit (m/s)')

# Zeichnen der Geschwindigkeit für Spieler 2 im zweiten Subplot
axs[1,1].plot(data.index, data['geschwindigkeit_p2_rollingMean'], color='green')
axs[1,1].set_title('Geschwindigkeit Spieler 2, Rolling Mean')
axs[1,1].set_xlabel('Frames (0.5 Sekunden)')
axs[1,1].set_ylabel('Geschwindigkeit (m/s)')

# Anpassung des Layouts für eine bessere Darstellung
plt.tight_layout()

# Anzeigen der Diagramme
plt.show()

In [ ]:
data["geschwindigkeit_p1_rm_ip"] = data["geschwindigkeit_p1_rollingMean"].interpolate()
data["geschwindigkeit_p2_rm_ip"] = data["geschwindigkeit_p2_rollingMean"].interpolate()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
# Zeichnen der Geschwindigkeit für Spieler 1 im ersten Subplot
axs[0,0].plot(data.index, data['geschwindigkeit_p1_rollingMean'], color='blue')
axs[0,0].set_title('Geschwindigkeit Spieler 1, Rolling Mean')
axs[0,0].set_xlabel('Frames (0.5 Sekunden)')
axs[0,0].set_ylabel('Geschwindigkeit (m/s)')

# Zeichnen der Geschwindigkeit für Spieler 2 im zweiten Subplot
axs[1,0].plot(data.index, data['geschwindigkeit_p2_rollingMean'], color='green')
axs[1,0].set_title('Geschwindigkeit Spieler 2, Rolling Mean')
axs[1,0].set_xlabel('Frames (0.5 Sekunden)')
axs[1,0].set_ylabel('Geschwindigkeit (m/s)')

# Zeichnen der Geschwindigkeit für Spieler 1 im ersten Subplot
axs[0,1].plot(data.index, data['geschwindigkeit_p1_rm_ip'], color='blue')
axs[0,1].set_title('Geschwindigkeit Spieler 1, Rolling Mean interpoliert')
axs[0,1].set_xlabel('Frames (0.5 Sekunden)')
axs[0,1].set_ylabel('Geschwindigkeit (m/s)')

# Zeichnen der Geschwindigkeit für Spieler 2 im zweiten Subplot
axs[1,1].plot(data.index, data['geschwindigkeit_p2_rm_ip'], color='green')
axs[1,1].set_title('Geschwindigkeit Spieler 2, Rolling Mean interpoliert')
axs[1,1].set_xlabel('Frames (0.5 Sekunden)')
axs[1,1].set_ylabel('Geschwindigkeit (m/s)')

# Anpassung des Layouts für eine bessere Darstellung
plt.tight_layout()

# Anzeigen der Diagramme
plt.show()

In [ ]:

cmap = ListedColormap(['green', 'red'])
# Hinzufügen von benutzerdefinierten Patches für die Legende
aktiver_patch = mpatches.Patch(color='green', label='Aktiv')
inaktiver_patch = mpatches.Patch(color='red', label='Inaktiv')
data['Status_color'] = data['Status'].map({'Aktiv': 1, 'Inaktiv': 0})
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

# Zeichnen der Geschwindigkeit für Spieler 1 im ersten Subplot
axs[0].plot(data.index, data['geschwindigkeit_p1_rm_ip'], color='blue')
axs[0].pcolorfast(axs[0].get_xlim(), axs[0].get_ylim(),
              data['Status_color'].values[np.newaxis],
              cmap=cmap, alpha=0.4)
axs[0].legend(handles=[aktiver_patch, inaktiver_patch])
axs[0].set_title('Geschwindigkeit Spieler 1, Rolling Mean interpoliert')
axs[0].set_xlabel('Zeit (Sekunden)')
axs[0].set_ylabel('Geschwindigkeit (m/s)')

# Zeichnen der Geschwindigkeit für Spieler 2 im zweiten Subplot
axs[1].plot(data.index, data['geschwindigkeit_p2_rm_ip'], color='yellow')
axs[1].set_title('Geschwindigkeit Spieler 2')
axs[1].set_xlabel('Zeit (ms)')
axs[1].set_ylabel('Geschwindigkeit (m/s)')
axs[1].pcolorfast(axs[1].get_xlim(), axs[1].get_ylim(),
              data['Status_color'].values[np.newaxis],
              cmap=cmap, alpha=0.4)
axs[1].legend(handles=[aktiver_patch, inaktiver_patch])

# Anpassung des Layouts für eine bessere Darstellung
plt.tight_layout()

# Anzeigen der Diagramme
plt.show()

In [ ]:
data2 = data[['geschwindigkeit_p1', 'geschwindigkeit_p2']]
data2

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

# Geschwindigkeiten von Spieler 1 und Spieler 2
geschwindigkeit_p1 = data['geschwindigkeit_p1_rm_ip'].dropna()
geschwindigkeit_p2 = data['geschwindigkeit_p2_rm_ip'].dropna()
frames = data['frame'][:len(geschwindigkeit_p1)]  # Ensure frames and geschwindigkeit_p1 have the same length

# Zeichnen der Geschwindigkeit für Spieler 1 im ersten Subplot
axs[0,0].plot(frames, geschwindigkeit_p1, color='blue')
axs[0,0].set_title('Geschwindigkeit Spieler 1, Rolling Mean interpoliert')
axs[0,0].set_xlabel('Frames (0.5 Sekunden)')
axs[0,0].set_ylabel('Geschwindigkeit (m/s)')

# Zeichnen der Geschwindigkeit für Spieler 2 im zweiten Subplot
axs[1,0].plot(frames, geschwindigkeit_p2, color='green')
axs[1,0].set_title('Geschwindigkeit Spieler 2, Rolling Mean interpoliert')
axs[1,0].set_xlabel('Frames (0.5 Sekunden)')
axs[1,0].set_ylabel('Geschwindigkeit (m/s)')

# Zeichnen der ACF für Spieler 1
plot_acf(geschwindigkeit_p1, ax=axs[0,1], lags= range(0,3000))
axs[0,1].set_title('ACF Spieler 1')

# Zeichnen der PACF für Spieler 1
plot_pacf(geschwindigkeit_p1, ax=axs[0,2],lags=range(0,3000) )
axs[0,2].set_title('PACF Spieler 1')

# Zeichnen der ACF für Spieler 2
plot_acf(geschwindigkeit_p2, ax=axs[1,1], lags= range(0,3000))
axs[1,1].set_title('ACF Spieler 2')

# Zeichnen der PACF für Spieler 2
plot_pacf(geschwindigkeit_p2, ax=axs[1,2], lags= range(0,3000))
axs[1,2].set_title('PACF Spieler 2')

plt.tight_layout()
plt.show()


In [ ]:
def fourier_analyse(df, column_names, frame_rate):
    if len(column_names) != 2:
        raise ValueError("Bitte genau zwei Spaltennamen angeben.")

    fig, axs = plt.subplots(2, 1, figsize=(10, 6))

    for i, column_name in enumerate(column_names):
        # Remove NAs
        df_fourier = df.dropna(subset=[column_name])

        # Fourier Analysis
        fft_values = np.fft.rfft(df_fourier[column_name])
        amplitudes = np.abs(fft_values)
        freqs = np.fft.rfftfreq(len(df_fourier[column_name]), d=1/frame_rate)

        valid_indices = freqs > 0
        freqs = freqs[valid_indices]
        amplitudes = amplitudes[valid_indices]

        # Plot the Fourier Transform for each column
        axs[i].plot(1/freqs, amplitudes)
        axs[i].set_title(f'Fourier Transformation of {column_name}')
        axs[i].set_xlabel("Seconds per Cycle (s)")
        axs[i].set_ylabel("Amplitude")
        axs[i].set_xlim(0, 200)  # Adjust as needed

    plt.tight_layout()
    plt.show()
fourier_analyse(data, ["geschwindigkeit_p1_rm_ip", "geschwindigkeit_p2_rm_ip"], 30)